In [ ]:
import sys 
sys.path.append('..')
import random
from PIL import Image
import requests
from utils import *
import yaml
from diffusers.utils import load_image, make_image_grid 
import numpy as np 
from ultralytics import YOLO
from scripts.sam_results import SAMResults
from datasets import load_dataset
import random

yaml_file = "../configs/fashion_people_detection.yml"
with open(yaml_file, "r") as f:
    config = yaml.safe_load(f)

labels_dict = config.get("names")
labels_dict_reversed = {v: k for k, v in labels_dict.items()}

In [ ]:
dataset_id = "jordandavis/fashion_people_detections"

ds = load_dataset(dataset_id, split='train', trust_remote_code=True, cache_dir='../hf_cache')

In [ ]:
model_path = "/workspace/SEGMENT/human_parsing/train2/weights/best.pt"
model = YOLO(model_path, task="segment")

In [ ]:
row_num = random.randint(0, len(ds))
print(f"Row Num: {row_num}")
row = ds[row_num]
image = row["image"]
image = resize_image_pil(image)
image.resize((256, 256))

In [ ]:
url = "https://pbs.twimg.com/media/BorTTDoIIAAqFx5.jpg"
image = load_image(url)
# image = resize_image_pil(image, 640)

In [ ]:
imgsz = max(image.size)

results = model(image, retina_masks=True, imgsz=imgsz)
result = results[0]

In [ ]:
def unload(result):
    masks = result.masks.data
    boxes = result.boxes.xyxy
    scores = result.boxes.conf

    labels = result.boxes.cls
    labels = [int(label.item()) for label in labels]
    phrases = [labels_dict[label] for label in labels]
    return masks, boxes, scores, phrases

In [ ]:
masks, boxes, scores, phrases = unload(result)

In [ ]:
r = SAMResults(image, labels_dict_reversed, masks=masks, boxes=boxes, scores=scores, phrases=phrases, person_masks_only=False )

In [ ]:
r.display_results()

In [ ]:
row = r.get_mask("phone")[0]
box = row.get('box')
mask = row.get("mask")
overlay = overlay_mask(image, mask, opacity=0.9)
overlay.crop(box).resize((512,512))